In [685]:
import pandas as pd 
from optbinning import OptimalBinning
from sklearn.impute import SimpleImputer
from sklearn.tree import DecisionTreeClassifier
import warnings
warnings.filterwarnings('ignore')

In [686]:
df = pd.read_csv('gen_data.csv')

### Tính đầy đủ 

In [687]:
pd.set_option('display.max_rows', None)

Tỷ lệ missing theo cột

In [688]:
missing_col = pd.DataFrame({
    'missing_count': df.isna().sum(),
    'missing_pct': df.isna().mean() * 100
}).sort_values('missing_pct', ascending=False)

missing_col

,missing_count,missing_pct
SOCIF,0,0.00
PURCOD_MIN,0,0.00
FLAG_DEPOSIT,0,0.00
CBAL_SHORTTERM_LOAN,0,0.00
CBAL_LONGTERM_LOAN,0,0.00
HAS_SHORTTERM_LOAN,0,0.00
HAS_LONGTERM_LOAN,0,0.00
DURATION_MAX,0,0.00
REMAINING_DURATION_MAX,0,0.00
TIME_TO_OP_MAX,0,0.00


In [689]:
pd.set_option('display.max_rows', 20)

Tỷ lệ missing theo row

In [690]:
df['ROW_MISSING_PCT'] = df.isna().mean(axis=1) * 100

df['ROW_MISSING_PCT'].describe()

count   1,138,327.00
mean            0.00
std             0.00
min             0.00
25%             0.00
50%             0.00
75%             0.00
max             0.00
Name: ROW_MISSING_PCT, dtype: float64

In [691]:
df['FLAG_HIGH_MISSING'] = (df['ROW_MISSING_PCT'] > 40).astype(int)

### Tính duy nhất 

Trùng khóa logic (SOCIF – year)

In [692]:
dup_key = df.duplicated(subset=['SOCIF', 'year'], keep=False)

df.loc[dup_key].shape

(0, 52)

Trùng toàn bộ bản ghi (record duplicate)

In [693]:
dup_full = df.duplicated(keep=False)

dup_full.sum()


np.int64(0)

In [694]:
dup_rate = dup_full.mean() * 100
print(f"Tỷ lệ bản ghi trùng lặp: {dup_rate:.2f}%")


Tỷ lệ bản ghi trùng lặp: 0.00%


### Tính kịp thời 

Check dùng future information (leakage)

In [695]:
# BAD_NEXT_12M chỉ được missing ở năm cuối
leak_check = df[
    (df['year'] < df['year'].max()) &
    (df['BAD_NEXT_12M'].isna())
]

leak_check.shape


(0, 52)

Check continuity theo năm

In [696]:
year_count = df.groupby('year')['SOCIF'].nunique()
year_count


year
2018    200349
2019    204636
2020    155962
2021    160198
2022    206838
2023    210344
Name: SOCIF, dtype: int64

### Tính phù hợp 

In [697]:
rule_valid_age = (
    df['TUOI'].between(0, 120)
)

df.loc[~rule_valid_age].shape


(0, 52)

In [698]:
rule_valid_ltv = df['LTV'].between(0, 400)

df.loc[~rule_valid_ltv].shape


(0, 52)

In [699]:
rule_valid_duration = (
    (df['DURATION_MAX'] >= 0) &
    (df['REMAINING_DURATION_MAX'] >= 0)
)

df.loc[~rule_valid_duration].shape


(0, 52)

In [700]:
valid_gender = df['C_GIOITINH'].isin(['M', 'F', 'O'])
df.loc[~valid_gender].shape


(0, 52)

In [701]:
df['FLAG_INVALID'] = ~(
    rule_valid_age &
    rule_valid_ltv &
    rule_valid_duration &
    valid_gender
)


In [702]:
pd.set_option('display.max_columns', None)

In [703]:
df.describe()

,SOCIF,TRINHDO,SOHUUNHA,NHANVIENBIDV,INHERENT_RISK,year,BASE_AUM,CURRENT_RISK,TUOI,INCOME,CBAL,CBALORG,AFLIMT_MAX,AFLIMT_MIN,AFLIMT_AVG,CBAL_AVG,CBAL_MAX,CBAL_MIN,COLLATERAL_VALUE,LTV,N_AVG_DEPOSIT_12M,N_AVG_DEPOSIT_6M,N_AVG_DD_12M,N_AVG_CD_12M,FLAG_SALARY_ACC,FLAG_DEPOSIT,CBAL_SHORTTERM_LOAN,CBAL_LONGTERM_LOAN,HAS_SHORTTERM_LOAN,HAS_LONGTERM_LOAN,DURATION_MAX,REMAINING_DURATION_MAX,TIME_TO_OP_MAX,RATE_AVG,PURCOD_MAX,PURCOD_MIN,MAX_DPD_12M,MAX_DPD_12M_OBS,AVG_OD_DPD_12M,SUM_ALL_OD_12M,BAD_CURRENT,XULYNO,MAX_NHOMNOCIC,N_AVG_OVERDUE_CBAL_12M,CBAL_TO_INC_12MON,REAL_GDP_GROWTH_12M,BAD_NEXT_12M,ROW_MISSING_PCT,FLAG_HIGH_MISSING
count,"1,138,327.00","1,138,327.00","1,138,327.00","1,138,327.00","1,138,327.00","1,138,327.00","1,138,327.00","1,138,327.00","1,138,327.00","1,138,327.00","1,138,327.00","1,138,327.00","1,138,327.00","1,138,327.00","1,138,327.00","1,138,327.00","1,138,327.00","1,138,327.00","1,138,327.00","1,138,327.00","1,138,327.00","1,138,327.00","1,138,327.00","1,138,327.00","1,138,327.00","1,138,327.00","1,138,327.00","1,138,327.00","1,138,327.00","1,138,327.00","1,138,327.00","1,138,327.00","1,138,327.00","1,138,327.00","1,138,327.00","1,138,327.00","1,138,327.00","1,138,327.00","1,138,327.00","1,138,327.00","1,138,327.00","1,138,327.00","1,138,327.00","1,138,327.00","1,138,327.00","1,138,327.00","1,138,327.00","1,138,327.00","1,138,327.00"
mean,"1,199,961.60",1.96,0.37,0.01,-0.45,"2,020.53","15,577,392.66",-0.42,38.73,"24,644,597.89","356,585,070.48","392,278,108.46","771,166,906.92","694,050,215.77","732,608,561.09","356,515,468.90","428,407,616.71","285,212,374.72","752,532,209.87",132.67,"5,598,486.30","5,598,486.30","1,119,696.86","4,478,789.44",0.91,0.83,"178,116,568.63","178,468,501.85",1.00,1.00,91.39,47.52,43.87,8.21,4.60,4.60,24.39,24.39,6.86,48.34,0.00,0.00,1.62,"166,507,333.69",14.42,5.66,0.12,0.00,0.00
std,"115,440.68",0.82,0.48,0.10,0.61,1.77,"22,781,955.16",0.76,9.69,"10,337,725.44","253,432,718.91","280,044,694.87","357,276,408.51","321,548,767.66","339,412,588.09","258,218,905.26","310,051,887.44","206,575,124.21","967,671,702.03",88.01,"9,899,404.16","9,899,404.16","1,979,880.83","7,919,523.32",0.28,0.38,"178,836,718.92","178,708,929.60",0.00,0.00,66.65,44.66,42.11,2.02,2.54,2.54,24.04,24.04,8.19,52.00,0.00,0.00,0.49,"130,535,861.52",7.53,2.03,0.33,0.00,0.00
min,"1,000,000.00",1.00,0.00,0.00,-1.41,"2,018.00","49,727.00",-3.45,18.00,"8,434,659.00","200,882.00","237,960.00","240,489,089.00","216,440,180.00","228,464,634.00","221,060.00","265,272.00","176,848.00","39,781.00",0.03,"13,278.00","13,278.00","2,655.00","10,623.00",0.00,0.00,9.00,28.00,1.00,1.00,6.00,0.00,1.00,5.00,1.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,2.58,0.00,0.00,0.00
25%,"1,100,216.00",1.00,0.00,0.00,-0.91,"2,019.00","4,345,251.00",-0.95,32.00,"18,148,954.50","187,436,063.50","205,614,280.00","546,773,025.50","492,095,722.50","519,434,373.50","184,988,652.50","222,429,275.50","147,990,921.50","4,771,108.00",20.22,"1,350,469.50","1,350,469.50","270,093.50","1,080,376.00",1.00,1.00,"58,739,301.00","58,954,187.50",1.00,1.00,36.00,17.00,15.00,6.70,2.00,2.00,5.00,5.00,1.00,8.00,0.00,0.00,1.00,"83,059,301.50",8.75,2.91,0.00,0.00,0.00
50%,"1,199,889.00",2.00,0.00,0.00,-0.57,"2,021.00","8,787,907.00",-0.47,39.00,"22,166,739.00","293,442,580.00","322,339,312.00","682,666,499.00","614,399,849.00","648,533,174.00","291,110,424.00","349,895,246.00","232,888,339.00","14,826,793.00",200.00,"2,838,993.00","2,838,993.00","567,798.00","2,271,195.00",1.00,1.00,"127,646,860.00","128,123,248.00",1.00,1.00,60.00,31.00,29.00,8.11,6.00,6.00,16.00,16.00,4.00,29.00,0.00,0.00,2.00,"138,681,626.00",13.05,7.02,0.00,0.00,0.00
75%,"1,299,960.00",2.00,1.00,0.00,-0.11,"2,022.00","18,020,020.50",0.05,45.00,"27,198,264.00","450,082,570.00","495,113,255.00","873,329,773.50","785,996,796.00","829,663,284.50","449,965,528.00","540,671,852.00","359,972,422.00","2,005,339,463.00",200.00,"6,083,645.00","6,083,645.00","1,216,729.00","4,866,916.00",1.00,1.00,"23

In [704]:
pd.set_option('display.max_columns', 20)

In [705]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1138327 entries, 0 to 1138326
Data columns (total 53 columns):
 #   Column                  Non-Null Count    Dtype  
---  ------                  --------------    -----  
 0   SOCIF                   1138327 non-null  int64  
 1   C_GIOITINH              1138327 non-null  object 
 2   TRINHDO                 1138327 non-null  int64  
 3   TTHONNHAN               1138327 non-null  object 
 4   SOHUUNHA                1138327 non-null  int64  
 5   NHANVIENBIDV            1138327 non-null  int64  
 6   INHERENT_RISK           1138327 non-null  float64
 7   year                    1138327 non-null  int64  
 8   BASE_AUM                1138327 non-null  int64  
 9   CURRENT_RISK            1138327 non-null  float64
 10  TUOI                    1138327 non-null  int64  
 11  INCOME                  1138327 non-null  int64  
 12  CBAL                    1138327 non-null  int64  
 13  CBALORG                 1138327 non-null  int64  
 14  AF

In [706]:
cat_cols = df.select_dtypes(include=['object', 'category']).columns

for col in cat_cols:
    print(f"\n{'='*20} {col} {'='*20}")
    vc = df[col].value_counts(dropna=False)
    pct = df[col].value_counts(normalize=True, dropna=False) * 100
    print(pd.concat([vc, pct.rename('pct_%')], axis=1))


==================== C_GIOITINH ====================
             count  pct_%
C_GIOITINH               
F           560397  49.23
M           555439  48.79
O            22491   1.98

==================== TTHONNHAN ====================
            count  pct_%
TTHONNHAN               
Married    596357  52.39
Single     541970  47.61

==================== SAMPLE_TYPE ====================
              count  pct_%
SAMPLE_TYPE               
TRAIN        721145  63.35
OOT          210344  18.48
OOS          206838  18.17


### Tính chính xác

Tuổi khách hàng

In [707]:
rule_acc_age = df['TUOI'] >= 15

LTV thực tế

In [708]:
rule_acc_ltv = df['LTV'] < 400

Hạn mức – dư nợ

In [709]:
rule_acc_limit = df['AFLIMT_MAX'] >= df['CBAL']

Thu nhập – dư nợ

In [710]:
rule_acc_dti = df['CBAL'] <= df['INCOME'] * 20

CIC – DPD

In [711]:
rule_acc_cic = ~(
    (df['MAX_DPD_12M_OBS'] < 30) & (df['MAX_NHOMNOCIC'] > 1)
) & ~(
    (df['MAX_DPD_12M_OBS'] >= 90) & (df['MAX_NHOMNOCIC'] < 3)
)

Flag & tỷ lệ chính xác

In [712]:
df['FLAG_INACCURATE'] = ~(
    rule_acc_age &
    rule_acc_ltv &
    rule_acc_limit &
    rule_acc_dti &
    rule_acc_cic
)

accuracy_rate = 1 - df['FLAG_INACCURATE'].mean()
accuracy_rate

np.float64(0.565352486587773)

In [713]:
rule_acc_age.sum()

np.int64(1138327)

In [714]:
rule_acc_ltv.sum()

np.int64(1138327)

In [715]:
rule_acc_dti.sum()

np.int64(903090)

In [716]:
rule_acc_cic.sum()

np.int64(810097)

### Tính đồng nhất

CBAL vs cấu trúc kỳ hạn

In [717]:
# rule_cons_balance = (
#     df['CBAL'] ==
#     df['CBAL_SHORTTERM_LOAN']
#     + df['CBAL_MIDTERM_LOAN']
#     + df['CBAL_LONGTERM_LOAN']
# )

# df.loc[~rule_cons_balance].shape

Deposit breakdown

In [718]:
rule_cons_deposit = (
    abs(
        df['N_AVG_DEPOSIT_12M'] -
        (df['N_AVG_DD_12M'] + df['N_AVG_CD_12M'])
    ) <= 1
)
df.loc[~rule_cons_deposit].shape

(0, 54)

DPD logic

In [719]:
rule_cons_dpd = df['MAX_DPD_12M_OBS'] <= df['MAX_DPD_12M']

df.loc[~rule_cons_dpd].shape


(0, 54)

Flag tổng hợp consistency

In [720]:
df['FLAG_INCONSISTENT'] = ~(
    # rule_cons_balance &
    rule_cons_deposit &
    rule_cons_dpd
)

BẢNG TỔNG HỢP (RẤT NÊN CÓ)

In [721]:
quality_summary = pd.DataFrame({
    'Metric': [
        'Completeness',
        'Uniqueness',
        'Timeliness',
        'Validity',
        'Accuracy',
        'Consistency'
    ],
    'Issue_Rate_%': [
        df['FLAG_HIGH_MISSING'].mean() * 100,
        dup_full.mean() * 100,
        leak_check.shape[0] / len(df) * 100,
        df['FLAG_INVALID'].mean() * 100,
        df['FLAG_INACCURATE'].mean() * 100,
        df['FLAG_INCONSISTENT'].mean() * 100
    ]
})

quality_summary

,Metric,Issue_Rate_%
0,Completeness,0.00
1,Uniqueness,0.00
2,Timeliness,0.00
3,Validity,0.00
4,Accuracy,43.46
5,Consistency,0.00


In [722]:
df['FLAG_AGE_INVALID'] = (~rule_acc_age).astype(int)
df['FLAG_LTV_HIGH']    = (~rule_acc_ltv).astype(int)
df['FLAG_DTI_HIGH']    = (~rule_acc_dti).astype(int)
df['FLAG_CIC_LOW']     = (~rule_acc_cic).astype(int)
# df['FLAG_INCONSISTENT_DEPOSIT'] = (~rule_cons_deposit).astype(int)

In [723]:
import numpy as np

In [ ]:
flag_cols = [c for c in df.columns if c.startswith('FLAG_')]

dq_flag_impact = []

for c in flag_cols:
    tmp = df.groupby(c)['BAD_NEXT_12M'].mean()
    dq_flag_impact.append({
        'FLAG': c,
        'BAD_RATE_0': tmp.get(0, np.nan),
        'BAD_RATE_1': tmp.get(1, np.nan),
        'LIFT': tmp.get(1, np.nan) / tmp.get(0, np.nan)
    })

dq_flag_impact = pd.DataFrame(dq_flag_impact).sort_values('LIFT', ascending=False)
dq_flag_impact

,FLAG,BAD_RATE_0,BAD_RATE_1,LIFT
8,FLAG_DTI_HIGH,0.11,0.15,1.29
4,FLAG_INACCURATE,0.14,0.10,0.69
0,FLAG_SALARY_ACC,0.17,0.12,0.66
9,FLAG_CIC_LOW,0.15,0.05,0.36
1,FLAG_DEPOSIT,0.28,0.09,0.31
2,FLAG_HIGH_MISSING,0.12,NaN,NaN
3,FLAG_INVALID,0.12,NaN,NaN
5,FLAG_INCONSISTENT,0.12,NaN,NaN
6,FLAG_AGE_INVALID,0.12,NaN,NaN
7,FLAG_LTV_HIGH,0.12,NaN,NaN


In [725]:
for f in ['FLAG_HIGH_MISSING', 'FLAG_INCONSISTENT']:
    print(f, df[f].value_counts(dropna=False))


FLAG_HIGH_MISSING FLAG_HIGH_MISSING
0    1138327
Name: count, dtype: int64
FLAG_INCONSISTENT FLAG_INCONSISTENT
False    1138327
Name: count, dtype: int64


In [726]:
df.shape

(1138327, 59)

In [727]:
df.drop(['FLAG_DTI_HIGH', 'FLAG_INACCURATE', 'FLAG_CIC_LOW', 'FLAG_HIGH_MISSING', 'FLAG_INCONSISTENT', 'ROW_MISSING_PCT'], axis=1, inplace=True)

In [728]:
df.columns

Index(['SOCIF', 'C_GIOITINH', 'TRINHDO', 'TTHONNHAN', 'SOHUUNHA',
       'NHANVIENBIDV', 'INHERENT_RISK', 'year', 'BASE_AUM', 'CURRENT_RISK',
       'TUOI', 'INCOME', 'CBAL', 'CBALORG', 'AFLIMT_MAX', 'AFLIMT_MIN',
       'AFLIMT_AVG', 'CBAL_AVG', 'CBAL_MAX', 'CBAL_MIN', 'COLLATERAL_VALUE',
       'LTV', 'N_AVG_DEPOSIT_12M', 'N_AVG_DEPOSIT_6M', 'N_AVG_DD_12M',
       'N_AVG_CD_12M', 'FLAG_SALARY_ACC', 'FLAG_DEPOSIT',
       'CBAL_SHORTTERM_LOAN', 'CBAL_LONGTERM_LOAN', 'HAS_SHORTTERM_LOAN',
       'HAS_LONGTERM_LOAN', 'DURATION_MAX', 'REMAINING_DURATION_MAX',
       'TIME_TO_OP_MAX', 'RATE_AVG', 'PURCOD_MAX', 'PURCOD_MIN', 'MAX_DPD_12M',
       'MAX_DPD_12M_OBS', 'AVG_OD_DPD_12M', 'SUM_ALL_OD_12M', 'BAD_CURRENT',
       'XULYNO', 'MAX_NHOMNOCIC', 'N_AVG_OVERDUE_CBAL_12M',
       'CBAL_TO_INC_12MON', 'REAL_GDP_GROWTH_12M', 'BAD_NEXT_12M',
       'SAMPLE_TYPE', 'FLAG_INVALID', 'FLAG_AGE_INVALID', 'FLAG_LTV_HIGH'],
      dtype='object')

In [729]:
df_train = df[df['SAMPLE_TYPE'] == 'TRAIN']
df_train_y = df_train['BAD_NEXT_12M']
df_OOS = df[df['SAMPLE_TYPE'] == 'OOS']
df_OOS_y = df_OOS['BAD_NEXT_12M']
df_OOT = df[df['SAMPLE_TYPE'] == 'OOT']
df_OOT_y = df_OOT['BAD_NEXT_12M']

In [730]:
df_train.drop(['SAMPLE_TYPE','BAD_NEXT_12M'], axis=1, inplace=True)
df_OOS.drop(['SAMPLE_TYPE', 'BAD_NEXT_12M'], axis=1, inplace=True)
df_OOT.drop(['SAMPLE_TYPE', 'BAD_NEXT_12M'], axis=1, inplace=True)

In [731]:
df_train.columns

Index(['SOCIF', 'C_GIOITINH', 'TRINHDO', 'TTHONNHAN', 'SOHUUNHA',
       'NHANVIENBIDV', 'INHERENT_RISK', 'year', 'BASE_AUM', 'CURRENT_RISK',
       'TUOI', 'INCOME', 'CBAL', 'CBALORG', 'AFLIMT_MAX', 'AFLIMT_MIN',
       'AFLIMT_AVG', 'CBAL_AVG', 'CBAL_MAX', 'CBAL_MIN', 'COLLATERAL_VALUE',
       'LTV', 'N_AVG_DEPOSIT_12M', 'N_AVG_DEPOSIT_6M', 'N_AVG_DD_12M',
       'N_AVG_CD_12M', 'FLAG_SALARY_ACC', 'FLAG_DEPOSIT',
       'CBAL_SHORTTERM_LOAN', 'CBAL_LONGTERM_LOAN', 'HAS_SHORTTERM_LOAN',
       'HAS_LONGTERM_LOAN', 'DURATION_MAX', 'REMAINING_DURATION_MAX',
       'TIME_TO_OP_MAX', 'RATE_AVG', 'PURCOD_MAX', 'PURCOD_MIN', 'MAX_DPD_12M',
       'MAX_DPD_12M_OBS', 'AVG_OD_DPD_12M', 'SUM_ALL_OD_12M', 'BAD_CURRENT',
       'XULYNO', 'MAX_NHOMNOCIC', 'N_AVG_OVERDUE_CBAL_12M',
       'CBAL_TO_INC_12MON', 'REAL_GDP_GROWTH_12M', 'FLAG_INVALID',
       'FLAG_AGE_INVALID', 'FLAG_LTV_HIGH'],
      dtype='object')